<a href="https://colab.research.google.com/github/Seetureddy/RAG_project/blob/main/Main_RAG_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain langchain-community faiss-cpu llama-cpp-python sentence-transformers pypdf


In [ ]:
!pip install --upgrade bitsandbytes
!pip install --upgrade accelerate transformers
!pip install --upgrade sentence-transformers


In [ ]:
import os
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, TextLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

In [ ]:
from transformers import BitsAndBytesConfig
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer


In [ ]:
from huggingface_hub import login

login()  # This will prompt you to enter your HF token


In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
documents = []

pdf_loader = PyPDFLoader("/content/drive/MyDrive/SurveySR_DD.pdf")  # Replace with your file path
text_loader = TextLoader("/content/drive/MyDrive/book.txt")  # Replace with your file path

documents.extend(pdf_loader.load())
documents.extend(text_loader.load())

In [ ]:
chunk_size = 500
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

chunks = []
for doc in documents:
    doc_chunks = text_splitter.split_text(doc.page_content)
    chunks.extend(doc_chunks)

In [ ]:
chunks = []
for doc in documents:
    doc_chunks = text_splitter.split_text(doc.page_content)  # Split text into chunks
    chunks.extend(doc_chunks)
    print(f"\nDocument split into {len(doc_chunks)} chunks")
    print("Example Chunk:\n", doc_chunks[0][:500])  # Show first chunk for reference

print("\nTotal Chunks Created:", len(chunks))

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(chunks, embedding_model)
retriever = vectorstore.as_retriever()

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

In [ ]:
def ask_query(query):
    retrieved_docs = retriever.get_relevant_documents(query)  # Get relevant docs

    #  Print Retrieved Documents
    print("\n Retrieved Documents:")
    for doc in retrieved_docs:
        print(doc.page_content[:500])  # Print first 500 chars of each doc

    if retrieved_docs:
        print("\n Retrieval successful. Generating answer from documents...")
        response = qa_chain.invoke({"query": query})
        return response["result"]

    else:
        print("\n No relevant documents found. Using LLM's general knowledge.")

        fallback_prompt = (
            "You are an intelligent AI assistant. If no relevant documents are found, "
            "answer the question using your general knowledge. Be confident and factual. "
            "Do NOT say 'I don’t know'. Provide a reasonable answer.\n\n"
            f"Question: {query}\n\nAnswer:"
        )

        response = llm.invoke(fallback_prompt)  # Call the LLM directly
        return response


In [ ]:
query = "What is the primary topic discussed in the eBook?"
answer = ask_query(query)
print("\nAnswer:", answer)

In [ ]:
query = "Who is the first president of the United States?"
response = llm.invoke(f"You are a knowledgeable AI. Answer this question:\n\n{query}")
print(response)


In [ ]:
query = "Who is the fprime minister of India?"
response = llm.invoke(f"You are a knowledgeable AI. Answer this question:\n\n{query}")
print(response)


In [ ]:
query = "What is the national bird of Korea?"
response = llm.invoke(f"You are a knowledgeable AI. Answer this question:\n\n{query}")
print(response)


In [ ]:
query = "What are the colors in indian national flag?"
response = llm.invoke(f"You are a knowledgeable AI. Answer this question:\n\n{query}")
print(response)


In [ ]:
query = "Under what license is this eBook available?"
answer = ask_query(query)
print("\nAnswer:", answer)
